# Language of Music - Preprocessing

This notebook describes and walks through the process of cleaning and normalizing lyrics for use in topic modelling.

It takes place in two parts:
- Loading the data
- Preprocessing the data

## Loading the Data

To load and manipulate the data, stored in a .csv format, I used Pandas DataFrames. This seems like the obvious choice when managing and manipulating data stored in a tabular format. It allows the user to open .csv files of various encodings, manage encoding errors, and directly access values within specific cells of a dataframe to manipulate, for a variety of data types.

Although this first part is specifically for loading the data, I also conducted a little bit of preliminary investigation into the raw data, to gain an understanding of what some of the steps might be required in preprocessing.

To start off, I import pandas.

In [ ]:
# Import Pandas
import pandas as pd

Next, I use the `.read_csv()` function to read the dataset into a `DataFrame` object from the .csv file it is stored in.

*Please note: A sample of the full dataset has been used in the repository for brevity. Full dataset size is > 9GB*

In [ ]:
# Read .csv as DataFrame object
df = pd.read_csv("./data/raw/song_lyrics_sampled.csv")